# Importing Libraries

In [54]:
# !pip install mne

import mne
import scipy.io
import numpy as np
import pathlib




# Importing Data

In [ ]:
# Get an array of file paths of each subject
directory = pathlib.Path('data')
files = directory.glob('*.mat')
file_paths = sorted([file for file in files if file.is_file()])

# Data Generation
Define functions, that take in a file_path, and return processed data, figures, etc.

In [62]:
def process_mat_file(file_path):
    mat = scipy.io.loadmat(file_path)

    channel_labels = np.array([mat['chann'][0][i][0] for i in range(len(mat['chann'][0]))])

    EEG_channel_labels = channel_labels[0:32]
    EEG_data = mat['X'][0:32, :]

    I = mat['I']

    print(I.T.shape)

process_mat_file(file_paths[0])

(245760, 6)
